In [1]:
import os
import time
import pickle
import numpy as np
import pandas as pd
from docx import Document
# from docx import Document as docx
from docx.shared import Pt

import docx2pdf
# from spire.doc import *
# from spire.doc.common import *

In [2]:
l=pd.concat([pd.read_csv('buyers.csv'),pd.read_csv('supplies.csv')], axis=0, ignore_index=True).to_dict('records')

In [3]:
with open('final_gstr3b_params.pkl', 'rb') as f:
    x1=pickle.load(f)
gstr3b={i[1]:i[3] for i in x1}

with open('final_gstr1_params.pkl', 'rb') as f:
    x2=pickle.load(f)
gstr1={i[1]:i[3] for i in x2}

In [4]:
# Randomized Dictionary's d 0th element

# k=0
# for i in l[0].keys():
#     l[0][i]=k
#     k=k+1
# l[0]

In [5]:
# for m in range(2):

#     x=l[m]

#     d_={}
#     for i in x.keys():
#         l_=[k for k, val in gstr3b.items() if val==i]
#         if len(l_)!=0:
#             d_.update(dict(zip(l_,tuple([x[i]])*len(l_))))
#     for i in set(gstr3b.keys()).difference(set(d_.keys())):
#         d_[i]='?'

#     d1_={}
#     for i in x.keys():
#         l_=[k for k, val in gstr1.items() if val==i]
#         if len(l_)!=0:
#             d1_.update(dict(zip(l_,tuple([x[i]])*len(l_))))
#     for i in set(gstr1.keys()).difference(set(d1_.keys())):
#         d1_[i]='?'

#     gstr3b_vals=[(k,v)for k,v in d_.items()]
#     gstr1_vals=[(k,v)for k,v in d1_.items()]    

#     f='gstr3b'
#     fname=f'{f}.docx'

#     doc=Document(fname)
#     all_tables=[]
#     for table in doc.tables:
#         rows=len(table.rows)
#         cols=len(table.columns)
#         data=np.empty((rows,cols), dtype=object)
#         for i,row in enumerate(table.rows):
#             for j,cell in enumerate(row.cells):
#                 data[i,j]=cell.text.strip()
#         all_tables.append(data)

#     for i,j in gstr3b_vals:
#         update_document(i[0], i[1], j, i[2])

#     doc.save(f"{f}_Updated_Vals_{m}.docx")
#     docx2pdf.convert(f"{f}_Updated_Vals_{m}.docx", f"{f}_Updated_{m}.pdf")
#     os.remove(f"{f}_Updated_Vals_{m}.docx")

#     f='gstr1'
#     fname=f'{f}.docx'

#     doc=Document(fname)
#     all_tables=[]
#     for table in doc.tables:
#         rows=len(table.rows)
#         cols=len(table.columns)
#         data=np.empty((rows,cols), dtype=object)
#         for i,row in enumerate(table.rows):
#             for j,cell in enumerate(row.cells):
#                 data[i,j]=cell.text.strip()
#         all_tables.append(data)

#     for i,j in gstr1_vals:
#         update_document(i[0], i[1], j, i[2])

#     doc.save(f"{f}_Updated_Vals_{m}.docx")
#     docx2pdf.convert(f"{f}_Updated_Vals_{m}.docx", f"{f}_Updated_{m}.pdf")
#     os.remove(f"{f}_Updated_Vals_{m}.docx")

In [7]:
import tqdm as tqdm

def update_document(row, col, new_value, table):
    if row >= len(table.rows) or col >= len(table.columns):
        print("Warning: The specified cell does not exist in the table.")
        return    
    cell = table.rows[row].cells[col]
    cell.text = str(new_value)
    for paragraph in cell.paragraphs:
        for run in paragraph.runs:
            run.font.name, run.font.size = 'Roboto', Pt(7.5)

def process_document(fname, vals, f, m):
    doc = Document(fname)
    all_tables = []
    for table in doc.tables:
        rows = len(table.rows)
        cols = len(table.columns)
        data = np.empty((rows, cols), dtype=object)
        for i, row in enumerate(table.rows):
            for j, cell in enumerate(row.cells):
                data[i, j] = cell.text.strip()
        all_tables.append(data)

    for i, j in vals:
        update_document(i[0], i[1], j, doc.tables[i[2]])

    output_docx = f"{f}_Updated_Vals_{m}.docx"
    doc.save(output_docx)
    # docx2pdf.convert(output_docx, f"{f}_Updated_{m}.pdf")
    os.remove(output_docx)

start_time=time.time()
for m in tqdm.tqdm(range(len(l))):
    x=l[m]
    d_={}
    d1_={}
    for gstr, d in [(gstr3b, d_), (gstr1, d1_)]:
        for i in x.keys():
            l_ = [k for k, val in gstr.items() if val == i]
            if len(l_) != 0:
                d.update(dict(zip(l_, tuple([x[i]]) * len(l_))))
        for i in set(gstr.keys()).difference(set(d.keys())):
            d[i] = '?'
    process_document('gstr3b.docx', [(k, v) for k, v in d_.items()], 'gstr3b', m)
    process_document('gstr1.docx', [(k, v) for k, v in d1_.items()], 'gstr1', m)
    
    # if m%10==0:
    #     print(f'Completed Batch {m}')
    # print(f"Time elapsed after iteration {m+1}: {time.time()-start_time}")
    # start_time=time.time()
end_time=time.time()
print(f"{(end_time-start_time)/60:.4f}minutes")

 13%|█▎        | 58/432 [01:05<07:01,  1.13s/it]


KeyboardInterrupt: 